In [15]:
import torch
import torchvision
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split


from scipy import stats
    

import math
import random
import pandas as pd


def countDiffMasks(mask1,mask2):
    total_diff=0
    for i in range(len(mask1)):
        m_1=mask1[i].flatten()
        m_2=mask2[i].flatten()
        count_same=(m_1 == m_2).sum()
        count_different=m_1.flatten().shape[0]-count_same
        total_diff+=count_different
    return total_diff


def get_mask_compression(mask_whole_model):
    num_total=0
    num_non_zeros=0
    for mask_each_layer in mask_whole_model:
        num_total+=torch.numel(mask_each_layer)
        num_non_zeros+=torch.count_nonzero(mask_each_layer)
        
    return (num_total-num_non_zeros)/num_total

    
    

def prune_model_get_mask(model,prune_rate):
    '''
    works purely on the model to get
    mask
    '''
    mask_whole_model=[]
    for nm, params in model.named_parameters():
        if "weight" in nm and "bn" not in nm and "linear" not in nm:
            mask_layer=torch.ones(params.shape)
#             print(nm,params.shape)
            abs_var=torch.std(torch.abs(params.data))
#             print(abs_var)
#             print(params)
            threshold=abs_var*prune_rate
            num_components=params.shape[0]
            for index_component in range(num_components):
                values=params[index_component]            
                re_shaped_values=values.flatten()                
                mask_vals = (torch.abs(re_shaped_values)>threshold).float()
                mask_vals=mask_vals.reshape(values.shape)
#                 print(mask_vals.shape)
                mask_layer[index_component]=mask_vals
            mask_whole_model.append(mask_layer)
    return mask_whole_model
 
    
def get_thresholds_each_layer(model,prune_rate):
    thresholds_per_layer=[]
    for nm, params in model.named_parameters():
        if "weight" in nm and "bn" not in nm and "linear" not in nm:
            mask_layer=torch.ones(params.shape)
            abs_std=torch.std(torch.abs(params.data))
            threshold=abs_std*prune_rate
            thresholds_per_layer.append(threshold)
    return thresholds_per_layer
    
                
def apply_mask_model(model,list_mask_whole_model):
    mask_layer_count=0
    for nm, params in model.named_parameters():
        if "weight" in nm and "bn" not in nm and "linear" not in nm:
            mask_layer=list_mask_whole_model[mask_layer_count]
            with torch.no_grad():
                device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#                 print("Devices are ",params.device,mask_layer.device)
                mask_layer=mask_layer.to(device)
    
                params.data=params.data*mask_layer            
            mask_layer_count+=1
    

def store_weights_in_dic(weight_description,model):
    for nm, params in model.named_parameters():
        if "weight" in nm and "bn" not in nm and "linear" not in nm:
            if nm not in weight_description:
                weight_description[nm]={}
            num_components=params.shape[0]
            for index_component in range(num_components):
                if index_component not in weight_description[nm]:
                    weight_description[nm][index_component]={}
                values=params[index_component]
                flat_values=values.flatten()
                for index_wt in range(flat_values.shape[0]):
                    if index_wt not in weight_description[nm][index_component]:
                        weight_description[nm][index_component][index_wt]=[]
                    weight_description[nm][index_component][index_wt].append(flat_values[index_wt].detach().item())
    return weight_description


def get_boolean_dict_weight_dict(weight_description,prune_rate,thresholds_per_layer):
    '''
    works on the dictionary of weights
    to create a dict of 1s and 0s to show
    how many times weight is more than threshold
    per layer
    '''
    boolean_weight_description={}
    count=0
    for layer in weight_description.keys():  
#         print("Count = ",count)
        threshold_this_layer=thresholds_per_layer[count]
#         print("Threshold for layer ",count,layer,"is ",threshold_this_layer)
        if layer not in boolean_weight_description:
            boolean_weight_description[layer]={}
        for index_component in weight_description[layer].keys():
            if index_component not in boolean_weight_description[layer]:
                boolean_weight_description[layer][index_component]={}
            for index_wt in weight_description[layer][index_component].keys():
                if index_wt not in boolean_weight_description[layer][index_component]:
                    boolean_weight_description[layer][index_component][index_wt]=[]
                all_wts=weight_description[layer][index_component][index_wt]
                all_wts_boolean=[]
                for wt in all_wts:
                    if abs(wt)>threshold_this_layer:
                        all_wts_boolean.append(1)
                    else:
                        all_wts_boolean.append(0)
                boolean_weight_description[layer][index_component][index_wt]=all_wts_boolean                    
        count+=1
        
    return boolean_weight_description
    
# create mask from boolean weight dictionary
def create_mask_from_boolean_wt(model,boolean_wt_dict):
    mask_whole_model=[]
    for nm, params in model.named_parameters():
        if "weight" in nm and "bn" not in nm and "linear" not in nm:
            mask_layer=torch.ones(params.shape)
#             print(nm,params.shape)
            abs_var=torch.var(torch.abs(params.data))
#             print(abs_var)
#             print(params)
#             threshold=abs_var*prune_rate
            num_components=params.shape[0]
            for index_component in range(num_components):
                values=params[index_component]            
                re_shaped_values=values.flatten() 
                mask_vals=[]
                for val_index in range(re_shaped_values.shape[0]):
                    boolean_vals=boolean_wt_dict[nm][index_component][val_index]
                    m = stats.mode(boolean_vals)
#                     print("Verdict for this weight is ",m[0][0])
                    mask_vals.append(m[0][0])
#                 mask_vals = (torch.abs(re_shaped_values)>threshold).float()                
                mask_vals=np.asarray(mask_vals)
                mask_vals=mask_vals.reshape(values.shape)
#                 print(mask_vals.shape)
                mask_layer[index_component]=torch.from_numpy(mask_vals)
            mask_whole_model.append(mask_layer)
    return mask_whole_model
         


In [16]:
# this part copied from shrinkbench

def nonzero(tensor):
    """Returns absolute number of values different from 0

    Arguments:
        tensor {numpy.ndarray} -- Array to compute over

    Returns:
        int -- Number of nonzero elements
    """
    return np.sum(tensor != 0.0)


def model_size(model, as_bits=False):
    """Returns absolute and nonzero model size

    Arguments:
        model {torch.nn.Module} -- Network to compute model size over

    Keyword Arguments:
        as_bits {bool} -- Whether to account for the size of dtype

    Returns:
        int -- Total number of weight & bias params
        int -- Out total_params exactly how many are nonzero
    """

    total_params = 0
    nonzero_params = 0
    for tensor in model.parameters():
        t = np.prod(tensor.shape)
        nz = nonzero(tensor.detach().cpu().numpy())
        if as_bits:
            bits = dtype2bits[tensor.dtype]
            t *= bits
            nz *= bits
        total_params += t
        nonzero_params += nz
    return int(total_params), int(nonzero_params)


In [17]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))        

def correct(output, target, topk=(1,)):
    """Computes how many correct outputs with respect to targets

    Does NOT compute accuracy but just a raw amount of correct
    outputs given target labels. This is done for each value in
    topk. A value is considered correct if target is in the topk
    highest values of output.
    The values returned are upperbounded by the given batch size

    [description]

    Arguments:
        output {torch.Tensor} -- Output prediction of the model
        target {torch.Tensor} -- Target labels from data

    Keyword Arguments:
        topk {iterable} -- [Iterable of values of k to consider as correct] (default: {(1,)})

    Returns:
        List(int) -- Number of correct values for each topk
    """

    with torch.no_grad():
        maxk = max(topk)
        # Only need to do topk for highest k, reuse for the rest
        _, pred = output.topk(k=maxk, dim=1, largest=True, sorted=True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(torch.tensor(correct_k.item()))
        return res


# below copied from shrinkbench, to be used later
# def accuracy(model, dataloader, topk=(1,)):
#     """Compute accuracy of a model over a dataloader for various topk

#     Arguments:
#         model {torch.nn.Module} -- Network to evaluate
#         dataloader {torch.utils.data.DataLoader} -- Data to iterate over

#     Keyword Arguments:
#         topk {iterable} -- [Iterable of values of k to consider as correct] (default: {(1,)})

#     Returns:
#         List(float) -- List of accuracies for each topk
#     """

#     # Use same device as model
#     device = next(model.parameters()).device

#     accs = np.zeros(len(topk))
#     with torch.no_grad():

#         for i, (input, target) in enumerate(dataloader):
#             input = input.to(device)
#             target = target.to(device)
#             output = model(input)

#             accs += np.array(correct(output, target, topk))

#     # Normalize over data length
#     accs /= len(dataloader.dataset)

#     return accs





# LENET 300-100 for MNIST and comparison
class FashionMnistNet(nn.Module):
    """Feedfoward neural network with 1 hidden layer"""
    def __init__(self):
        super(FashionMnistNet, self).__init__()
        self.fc1 = nn.Linear(28*28, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)        
        self.fc4 = nn.Linear(64, 10)
        self.fc4.is_classifier = True
        
    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))        
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)
    
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        top_1, top_5 = correct(out, labels,topk=(1,5))
#         print("Batch is ",batch[1].shape)
        
        top_1=top_1/batch[1].shape[0]
        top_5=top_5/batch[1].shape[0]

#         print("corr",top_1,top_5)
#         return {'val_loss': loss, 'val_acc': acc}
        return {'val_loss': loss, 'val_acc': acc, 'top_1': top_1, 'top_5': top_5}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        
        batch_top_1s = [x['top_1'] for x in outputs]
#         print(batch_top_1s)
        epoch_top_1 = torch.stack(batch_top_1s).mean()      # Combine top_1
        
        batch_top_5s = [x['top_5'] for x in outputs]
        epoch_top_5 = torch.stack(batch_top_5s).mean()      # Combine top_5
        
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item(),
               'val_top_1': epoch_top_1.item(), 'val_top_5': epoch_top_5.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}, val_top_1: {:.4f}, val_top_5: {:.4f}".format(
                                epoch, result['val_loss'], result['val_acc'], 
                                result['val_top_1'], result['val_top_5']))
        

In [18]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)



class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)
    
    
def evaluate(model, val_loader):
    """Evaluate the model's performance on the validation set"""
    outputs = [model.validation_step(batch) for batch in val_loader]
#     print("outputs are ",outputs)
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD,
        weight_description=None,mask_whole_model=None,
       model_state_path=None):
    """Train the model using gradient descent"""
    print("At train")
    history = []
    best_so_far=-999        
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            if mask_whole_model:
#                 print("Applying mask")
                apply_mask_model(model,mask_whole_model)
        # Validation phase
        result = evaluate(model, val_loader)
        if best_so_far<result["val_top_1"]:
            best_so_far=result["val_top_1"]
            if model_state_path:
                torch.save(model.state_dict(), model_state_path)
        
        model.epoch_end(epoch, result)
        history.append(result)
        print("wt desc = ",weight_description)
        if weight_description!=None:
            print("going for weight")
            weight_description=store_weights_in_dic(weight_description,model)
    return history, weight_description


def predict_image(img, model):
    xb = to_device(img.unsqueeze(0), device)
    yb = model(xb)
    _, preds  = torch.max(yb, dim=1)
    return preds[0].item()

In [5]:
print("Program with Song Han fashionmnist")

print("Torch cuda ",torch.cuda.is_available())


device = get_default_device()
print("device ",device)



dataset = FashionMNIST(root='data/', download=True, transform=ToTensor())


# Define test dataset
test_dataset = FashionMNIST(root='data/', train=False,transform=ToTensor())

val_size = 10000
train_size = len(dataset) - val_size

train_ds, val_ds = random_split(dataset, [train_size, val_size])

batch_size=128

train_loader = DataLoader(train_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size*2, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=256)

shape=dataset[0][0].shape
input_size=1
for s in shape:
    input_size*=s

    
    


num_classes = 10


Program with Song Han fashionmnist
Torch cuda  False
device  cpu


In [6]:
train_loader = DeviceDataLoader(train_loader, device)
val_loader = DeviceDataLoader(val_loader, device)
test_loader = DeviceDataLoader(test_loader, device)

targets=train_ds.dataset.targets
training_data=torch.tensor(train_ds.dataset.data)

training_data = training_data.to(device=device)


/tmp/ipykernel_3293965/3744492648.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  training_data=torch.tensor(train_ds.dataset.data)


In [7]:
model=FashionMnistNet()

history = [evaluate(model, val_loader)]
print("initial result",history)
# weight_description={}
epochs=20
lr=0.01
# history2,weight_description = fit(epochs, lr, model, train_loader, val_loader,weight_description=weight_description)



initial result [{'val_loss': 2.3034565448760986, 'val_acc': 0.06425781548023224, 'val_top_1': 0.06425781548023224, 'val_top_5': 0.5044921636581421}]


In [8]:




metrics={}
metrics["prune_rate"]=[]
metrics["compression"]=[]
metrics["epochs"]=[]
metrics["top_5"]=[]
metrics["top_1"]=[]

df_base=pd.read_csv("results_sheet/song_hn.csv")
done_pruning_list=list(df_base["prune_rate"])


# prune_rate_range=[0.1,0.3,0.4,0.8,1.1,1.3,1.5,1.7,
#                   1.8,2.1,2.4,2.9,3.1,3.15,3.12,3.22,3.29,3.35,3.4,3.45,3.5,3.55,
#                   3.6,3.65,3.7,3.75,3.8,3.85,3.9,3.95,4,4.25,4.5,5,6]

prune_rate_range=[4.51,4.53,4.55,4.57,5.01,5.03,5.05,5.07,5.1,5.3,5.5,5.7,5.9,5.9]

for prune_rate in prune_rate_range:
    if prune_rate in done_pruning_list:
        continue

    model_state_path="model_state/mod.pt"
    if torch.cuda.is_available():
        model.load_state_dict(torch.load(model_state_path))
    else:
        model.load_state_dict(torch.load(model_state_path,map_location=torch.device('cpu')))
    


    result = evaluate(model, test_loader)
    print("Test result is ",result)


    total_size,nz_size=model_size(model)
    compression=(total_size-nz_size)/total_size
    print("Original Compression=",compression)

    
    mask_whole_model=prune_model_get_mask(model,prune_rate)

    print("Mask compression = ",prune_rate,get_mask_compression(mask_whole_model))


    apply_mask_model(model,mask_whole_model)


    epochs=30
    history_prune,_ = fit(epochs, lr, model, train_loader, val_loader,
                          mask_whole_model=mask_whole_model)


    total_size,nz_size=model_size(model)
    compression=(total_size-nz_size)/total_size
    res = evaluate(model, test_loader)

    print("Compression=",compression,"Result after pruning and retraining is ",res)
    metrics["prune_rate"].append(prune_rate)
    metrics["compression"].append(compression)
    metrics["epochs"].append(epochs)
    metrics["top_5"].append(res['val_top_5'])
    metrics["top_1"].append(res['val_top_1'])




Test result is  {'val_loss': 0.35806748270988464, 'val_acc': 0.871874988079071, 'val_top_1': 0.871874988079071, 'val_top_5': 0.9970703125}
Original Compression= 0.0
Mask compression =  4.51 tensor(0.9918)
At train
Epoch [0], val_loss: 2.2247, val_acc: 0.1933, val_top_1: 0.1933, val_top_5: 0.5932
wt desc =  None
Epoch [1], val_loss: 2.1195, val_acc: 0.2677, val_top_1: 0.2677, val_top_5: 0.7926
wt desc =  None
Epoch [2], val_loss: 1.9755, val_acc: 0.2396, val_top_1: 0.2396, val_top_5: 0.8080
wt desc =  None
Epoch [3], val_loss: 1.8737, val_acc: 0.2556, val_top_1: 0.2556, val_top_5: 0.8263
wt desc =  None
Epoch [4], val_loss: 1.8006, val_acc: 0.3021, val_top_1: 0.3021, val_top_5: 0.8563
wt desc =  None
Epoch [5], val_loss: 1.7427, val_acc: 0.3485, val_top_1: 0.3485, val_top_5: 0.8680
wt desc =  None
Epoch [6], val_loss: 1.6945, val_acc: 0.3612, val_top_1: 0.3612, val_top_5: 0.8693
wt desc =  None
Epoch [7], val_loss: 1.6547, val_acc: 0.4254, val_top_1: 0.4254, val_top_5: 0.8568
wt desc = 

Epoch [12], val_loss: 1.5385, val_acc: 0.4556, val_top_1: 0.4556, val_top_5: 0.8412
wt desc =  None
Epoch [13], val_loss: 1.5228, val_acc: 0.4670, val_top_1: 0.4670, val_top_5: 0.8484
wt desc =  None
Epoch [14], val_loss: 1.5097, val_acc: 0.4655, val_top_1: 0.4655, val_top_5: 0.8649
wt desc =  None
Epoch [15], val_loss: 1.4972, val_acc: 0.4720, val_top_1: 0.4720, val_top_5: 0.8631
wt desc =  None
Epoch [16], val_loss: 1.4861, val_acc: 0.4759, val_top_1: 0.4759, val_top_5: 0.8504
wt desc =  None
Epoch [17], val_loss: 1.4760, val_acc: 0.4757, val_top_1: 0.4757, val_top_5: 0.8546
wt desc =  None
Epoch [18], val_loss: 1.4652, val_acc: 0.4783, val_top_1: 0.4783, val_top_5: 0.8586
wt desc =  None
Epoch [19], val_loss: 1.4548, val_acc: 0.4804, val_top_1: 0.4804, val_top_5: 0.8564
wt desc =  None
Epoch [20], val_loss: 1.4444, val_acc: 0.4825, val_top_1: 0.4825, val_top_5: 0.8572
wt desc =  None
Epoch [21], val_loss: 1.4345, val_acc: 0.4797, val_top_1: 0.4797, val_top_5: 0.8623
wt desc =  None


Epoch [26], val_loss: 1.4554, val_acc: 0.4348, val_top_1: 0.4348, val_top_5: 0.8238
wt desc =  None
Epoch [27], val_loss: 1.4497, val_acc: 0.4345, val_top_1: 0.4345, val_top_5: 0.8274
wt desc =  None
Epoch [28], val_loss: 1.4439, val_acc: 0.4358, val_top_1: 0.4358, val_top_5: 0.8252
wt desc =  None
Epoch [29], val_loss: 1.4386, val_acc: 0.4358, val_top_1: 0.4358, val_top_5: 0.8274
wt desc =  None
Compression= 0.9938581820878062 Result after pruning and retraining is  {'val_loss': 1.424758791923523, 'val_acc': 0.4458984434604645, 'val_top_1': 0.4458984434604645, 'val_top_5': 0.82958984375}
Test result is  {'val_loss': 0.35806748270988464, 'val_acc': 0.871874988079071, 'val_top_1': 0.871874988079071, 'val_top_5': 0.9970703125}
Original Compression= 0.0
Mask compression =  5.03 tensor(0.9958)
At train
Epoch [0], val_loss: 2.2765, val_acc: 0.1851, val_top_1: 0.1851, val_top_5: 0.5551
wt desc =  None
Epoch [1], val_loss: 2.2352, val_acc: 0.1854, val_top_1: 0.1853, val_top_5: 0.6713
wt desc 

Epoch [22], val_loss: 1.4876, val_acc: 0.4323, val_top_1: 0.4323, val_top_5: 0.8245
wt desc =  None
Epoch [23], val_loss: 1.4790, val_acc: 0.4331, val_top_1: 0.4331, val_top_5: 0.8238
wt desc =  None
Epoch [24], val_loss: 1.4714, val_acc: 0.4347, val_top_1: 0.4347, val_top_5: 0.8729
wt desc =  None
Epoch [25], val_loss: 1.4640, val_acc: 0.4353, val_top_1: 0.4353, val_top_5: 0.8235
wt desc =  None
Epoch [26], val_loss: 1.4579, val_acc: 0.4332, val_top_1: 0.4332, val_top_5: 0.8261
wt desc =  None
Epoch [27], val_loss: 1.4516, val_acc: 0.4354, val_top_1: 0.4354, val_top_5: 0.8256
wt desc =  None
Epoch [28], val_loss: 1.4461, val_acc: 0.4380, val_top_1: 0.4380, val_top_5: 0.8237
wt desc =  None
Epoch [29], val_loss: 1.4426, val_acc: 0.4385, val_top_1: 0.4385, val_top_5: 0.8207
wt desc =  None
Compression= 0.9941424110857547 Result after pruning and retraining is  {'val_loss': 1.4269108772277832, 'val_acc': 0.44775390625, 'val_top_1': 0.44775390625, 'val_top_5': 0.8246093988418579}
Test res

Epoch [2], val_loss: 2.2964, val_acc: 0.0958, val_top_1: 0.0958, val_top_5: 0.5464
wt desc =  None
Epoch [3], val_loss: 2.2919, val_acc: 0.1107, val_top_1: 0.1107, val_top_5: 0.5522
wt desc =  None
Epoch [4], val_loss: 2.2876, val_acc: 0.1498, val_top_1: 0.1498, val_top_5: 0.5578
wt desc =  None
Epoch [5], val_loss: 2.2829, val_acc: 0.1656, val_top_1: 0.1656, val_top_5: 0.5634
wt desc =  None
Epoch [6], val_loss: 2.2771, val_acc: 0.1757, val_top_1: 0.1757, val_top_5: 0.5795
wt desc =  None
Epoch [7], val_loss: 2.2701, val_acc: 0.1792, val_top_1: 0.1792, val_top_5: 0.5804
wt desc =  None
Epoch [8], val_loss: 2.2618, val_acc: 0.1797, val_top_1: 0.1797, val_top_5: 0.5809
wt desc =  None
Epoch [9], val_loss: 2.2526, val_acc: 0.1807, val_top_1: 0.1807, val_top_5: 0.5691
wt desc =  None
Epoch [10], val_loss: 2.2428, val_acc: 0.1831, val_top_1: 0.1831, val_top_5: 0.5748
wt desc =  None
Epoch [11], val_loss: 2.2330, val_acc: 0.1687, val_top_1: 0.1687, val_top_5: 0.5756
wt desc =  None
Epoch [1

Epoch [17], val_loss: 2.2299, val_acc: 0.1739, val_top_1: 0.1739, val_top_5: 0.5694
wt desc =  None
Epoch [18], val_loss: 2.2218, val_acc: 0.1762, val_top_1: 0.1762, val_top_5: 0.5695
wt desc =  None
Epoch [19], val_loss: 2.2145, val_acc: 0.1780, val_top_1: 0.1780, val_top_5: 0.5748
wt desc =  None
Epoch [20], val_loss: 2.2079, val_acc: 0.1799, val_top_1: 0.1799, val_top_5: 0.5702
wt desc =  None
Epoch [21], val_loss: 2.2022, val_acc: 0.1807, val_top_1: 0.1807, val_top_5: 0.5748
wt desc =  None
Epoch [22], val_loss: 2.1967, val_acc: 0.1830, val_top_1: 0.1830, val_top_5: 0.5774
wt desc =  None
Epoch [23], val_loss: 2.1921, val_acc: 0.1842, val_top_1: 0.1842, val_top_5: 0.5271
wt desc =  None
Epoch [24], val_loss: 2.1875, val_acc: 0.1847, val_top_1: 0.1847, val_top_5: 0.5788
wt desc =  None
Epoch [25], val_loss: 2.1833, val_acc: 0.1849, val_top_1: 0.1849, val_top_5: 0.5786
wt desc =  None
Epoch [26], val_loss: 2.1796, val_acc: 0.1849, val_top_1: 0.1849, val_top_5: 0.5786
wt desc =  None


In [9]:
dataframe_results=pd.DataFrame(metrics)
dataframe_results=pd.concat([df_base, dataframe_results], ignore_index=True)

print(dataframe_results.head())

   prune_rate  compression  epochs     top_5     top_1
0         0.1     0.037283      30  0.996582  0.879395
1         0.3     0.110376      30  0.996973  0.880957
2         0.4     0.147058      30  0.996680  0.861621
3         0.8     0.292117      30  0.997266  0.871094
4         1.1     0.400215      30  0.996777  0.871582


In [ ]:
if os.path.isfile(os.path.join("results_sheet","song_hn.csv")):
    df_base=pd.read_csv(os.path.join("results_sheet","song_hn.csv"))
    df_concat=pd.concat([df_base,dataframe_results])
    dataframe_results=df_concat

dataframe_results.to_csv(os.path.join("results_sheet","song_hn.csv"),index=False)

In [11]:
l1=[0.55, 0.6,0.65, 0.7,0.75,0.8,0.85,0.9,0.95,
                 1.1, 1.4, 1.7, 2, 2.3,2.6,3,3.2,3.6,3.9,
                  4.3,4.6,4.9
                 ]
l2=[0.02, 0.025,0.03, 0.035, 0.04, 0.045, 0.05, 0.055, 0.06, 0.065,0.07, 0.09,0.1,
                 0.12, 0.14, 0.16, 0.18, 0.2, 0.22, 0.24, 0.26, 0.28, 0.29, 0.3, 0.5,1,5,9,12,14]

In [12]:
l1.extend(l2)

In [13]:
l1=sorted(list(set(l1)))

In [19]:
df_res=pd.read_csv("results_sheet/song_hn.csv")

In [21]:
df_res

,prune_rate,compression,epochs,top_5,top_1
0,0.10,0.037283,30,0.996582,0.879395
1,0.30,0.110376,30,0.996973,0.880957
2,0.40,0.147058,30,0.996680,0.861621
3,0.80,0.292117,30,0.997266,0.871094
4,1.10,0.400215,30,0.996777,0.871582
5,1.30,0.471433,30,0.996680,0.875000
6,1.50,0.541600,30,0.996484,0.879883
7,1.70,0.609729,30,0.996484,0.877930
8,1.80,0.642984,30,0.996680,0.872461
9,2.10,0.737723,30,0.996191,0.876758
